# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [7]:
# Helper functions
import itertools
from collections import deque

def is_location_safe(people_set):
    """Checks if a single location (bank, island, or boat) is safe."""
    husbands = {p for p in people_set if p.startswith('H')}
    wives = {p for p in people_set if p.startswith('W')}

    for wife in wives:
        wife_number = wife[1:]
        her_husband = f'H{wife_number}'
        # If her husband is not present...
        if her_husband not in people_set:
            # Check if any other husbands are present
            if any(h for h in husbands):
                return False # Jealousy rule violated
    return True

def is_valid_state(state, n):
    """Checks if the entire state (all locations) is valid."""
    _boat_pos, left_set, island_set, right_set = state
    
    # Check each bank/island
    if not is_location_safe(left_set):
        # print(f"Invalid state: Left bank unsafe {left_set}")
        return False
    if not is_location_safe(island_set):
        # print(f"Invalid state: Island unsafe {island_set}")
        return False
    if not is_location_safe(right_set):
        # print(f"Invalid state: Right bank unsafe {right_set}")
        return False
        
    return True
    
def get_people_at(state, location):
    """Helper to get the set of people at a specific location."""
    boat_pos, left_set, island_set, right_set = state
    if location == 'L':
        return left_set
    elif location == 'I':
        return island_set
    elif location == 'R':
        return right_set
    else:
        raise ValueError("Invalid location")

def get_next_states(state, n):
    """Generates all valid successor states."""
    boat_pos, left_set, island_set, right_set = state
    next_states = []
    
    current_bank_people = get_people_at(state, boat_pos)
    
    possible_destinations = []
    if boat_pos == 'L':
        possible_destinations.append('I')
    elif boat_pos == 'I':
        possible_destinations.append('L')
        possible_destinations.append('R')
    elif boat_pos == 'R':
        possible_destinations.append('I')
        
    # Try moving 1 or 2 people
    for num_people in [1, 2]:
        if len(current_bank_people) >= num_people:
            for people_in_boat_tuple in itertools.combinations(current_bank_people, num_people):
                people_in_boat = frozenset(people_in_boat_tuple)
                
                # Check if the boat itself is safe during transit
                if not is_location_safe(people_in_boat):
                    continue

                for dest_pos in possible_destinations:
                    # Create the new state
                    new_left = set(left_set)
                    new_island = set(island_set)
                    new_right = set(right_set)
                    
                    current_location_set = None
                    dest_location_set = None

                    if boat_pos == 'L': current_location_set = new_left
                    elif boat_pos == 'I': current_location_set = new_island
                    else: current_location_set = new_right
                        
                    if dest_pos == 'L': dest_location_set = new_left
                    elif dest_pos == 'I': dest_location_set = new_island
                    else: dest_location_set = new_right
                        
                    # Move people
                    current_location_set -= people_in_boat
                    dest_location_set.update(people_in_boat)

                    new_state = (dest_pos, frozenset(new_left), frozenset(new_island), frozenset(new_right))

                    # Check if the resulting state is valid
                    if is_valid_state(new_state, n):
                        next_states.append(new_state)
                        
    return next_states

In [8]:
# DFS

def solve_dfs(n):
    """Solves the Jealous Husbands problem using DFS."""
    all_people = frozenset({f'{p}{i+1}' for p in ['H', 'W'] for i in range(n)})
    
    initial_state = ('L', all_people, frozenset(), frozenset())
    goal_state_people = all_people
    
    stack = deque([(initial_state, [initial_state])]) # (state, path)
    visited = {initial_state}
    
    while stack:
        current_state, current_path = stack.pop()
        
        _boat_pos, _left, _island, right = current_state
        
        # Check if goal reached
        if right == goal_state_people:
            print(f"Solution found with {len(current_path) - 1} moves.")
            return current_path

        # Explore neighbors
        for next_state in get_next_states(current_state, n):
            if next_state not in visited:
                visited.add(next_state)
                new_path = current_path + [next_state]
                stack.append((next_state, new_path))
                
    print("No solution found.")
    return None

# --- Example Usage for DFS Approach ---
N_COUPLES = 2
solution_path = solve_dfs(N_COUPLES)

if solution_path:
    print("\n--- Solution Path ---")
    for i, state in enumerate(solution_path):
        boat, left, island, right = state
        print(f"Step {i}:")
        print(f"  Boat: {boat}")
        print(f"  Left Bank : {sorted(list(left))}")
        print(f"  Island    : {sorted(list(island))}")
        print(f"  Right Bank: {sorted(list(right))}")
        print("-" * 20)


Solution found with 16 moves.

--- Solution Path ---
Step 0:
  Boat: L
  Left Bank : ['H1', 'H2', 'W1', 'W2']
  Island    : []
  Right Bank: []
--------------------
Step 1:
  Boat: I
  Left Bank : ['H2', 'W2']
  Island    : ['H1', 'W1']
  Right Bank: []
--------------------
Step 2:
  Boat: R
  Left Bank : ['H2', 'W2']
  Island    : []
  Right Bank: ['H1', 'W1']
--------------------
Step 3:
  Boat: I
  Left Bank : ['H2', 'W2']
  Island    : ['H1']
  Right Bank: ['W1']
--------------------
Step 4:
  Boat: L
  Left Bank : ['H1', 'H2', 'W2']
  Island    : []
  Right Bank: ['W1']
--------------------
Step 5:
  Boat: I
  Left Bank : ['W2']
  Island    : ['H1', 'H2']
  Right Bank: ['W1']
--------------------
Step 6:
  Boat: R
  Left Bank : ['W2']
  Island    : []
  Right Bank: ['H1', 'H2', 'W1']
--------------------
Step 7:
  Boat: I
  Left Bank : ['W2']
  Island    : ['H1', 'W1']
  Right Bank: ['H2']
--------------------
Step 8:
  Boat: R
  Left Bank : ['W2']
  Island    : ['W1']
  Right Ban

In [9]:
# Backtracking

def backtrack_recursive(current_state, current_path, visited, n, goal_state_people):
    """Recursive helper function for backtracking search."""
    
    _boat_pos, _left, _island, right = current_state

    # Base Case: Goal reached
    if right == goal_state_people:
        print(f"Solution found with {len(current_path) - 1} moves.")
        return current_path

    # Mark state as visited for this path exploration
    visited.add(current_state)

    # Explore neighbors
    for next_state in get_next_states(current_state, n):
        if next_state not in visited:
            # Recursive call for the valid, unvisited neighbor
            result = backtrack_recursive(next_state, current_path + [next_state], visited, n, goal_state_people)
            
            # If a solution is found down this path, return it immediately
            if result is not None:
                return result
                
    # Backtrack: If no neighbor leads to a solution from this state
    # No explicit state removal needed here as visited prevents cycles/re-exploration.
    # The function returning None signifies backtracking.
    return None

def solve_backtracking(n):
    """Solves the Jealous Husbands problem using Backtracking."""
    all_people = frozenset({f'{p}{i+1}' for i in range(n) for p in ['H', 'W']})
    
    initial_state = ('L', all_people, frozenset(), frozenset())
    goal_state_people = all_people
    
    visited = set() # Keep track of visited states globally

    print("Starting Backtracking Search...")
    solution_path = backtrack_recursive(initial_state, [initial_state], visited, n, goal_state_people)
    
    if solution_path is None:
        print("No solution found.")
        
    return solution_path

# --- Example Usage for Backtracking Approach ---
N_COUPLES_BT = 2 
solution_path_bt = solve_backtracking(N_COUPLES_BT)

if solution_path_bt:
    print("\n--- Backtracking Solution Path ---")
    for i, state in enumerate(solution_path_bt):
        boat, left, island, right = state
        print(f"Step {i}:")
        print(f"  Boat: {boat}")
        print(f"  Left Bank : {sorted(list(left))}")
        print(f"  Island    : {sorted(list(island))}")
        print(f"  Right Bank: {sorted(list(right))}")
        print("-" * 20)

Starting Backtracking Search...
Solution found with 34 moves.

--- Backtracking Solution Path ---
Step 0:
  Boat: L
  Left Bank : ['H1', 'H2', 'W1', 'W2']
  Island    : []
  Right Bank: []
--------------------
Step 1:
  Boat: I
  Left Bank : ['H1', 'W1']
  Island    : ['H2', 'W2']
  Right Bank: []
--------------------
Step 2:
  Boat: L
  Left Bank : ['H1', 'H2', 'W1']
  Island    : ['W2']
  Right Bank: []
--------------------
Step 3:
  Boat: I
  Left Bank : ['H1', 'H2']
  Island    : ['W1', 'W2']
  Right Bank: []
--------------------
Step 4:
  Boat: L
  Left Bank : ['H1', 'H2', 'W2']
  Island    : ['W1']
  Right Bank: []
--------------------
Step 5:
  Boat: I
  Left Bank : ['H2', 'W2']
  Island    : ['H1', 'W1']
  Right Bank: []
--------------------
Step 6:
  Boat: R
  Left Bank : ['H2', 'W2']
  Island    : []
  Right Bank: ['H1', 'W1']
--------------------
Step 7:
  Boat: I
  Left Bank : ['H2', 'W2']
  Island    : ['H1']
  Right Bank: ['W1']
--------------------
Step 8:
  Boat: L
  Le

In [14]:
import heapq # Use heapq for efficient priority queue implementation

def heuristic(state, n):
    """Estimates the cost from the current state to the goal."""
    _boat_pos, left_set, island_set, _right_set = state
    # Number of people not on the right bank. 
    # Admissible because each person requires at least one more move segment.
    return len(left_set) + len(island_set) 

def solve_astar(n):
    """Solves the Jealous Husbands problem using A* search."""
    all_people = frozenset({f'{p}{i+1}' for p in ['H', 'W'] for i in range(n)})
    
    initial_state = ('L', all_people, frozenset(), frozenset())
    goal_state_people = all_people
    
    # Priority queue stores tuples: (f_cost, g_cost, state, path)
    # g_cost is the actual path cost so far
    # f_cost = g_cost + h_cost
    priority_queue = []
    
    initial_g_cost = 0
    initial_h_cost = heuristic(initial_state, n)
    initial_f_cost = initial_g_cost + initial_h_cost
    heapq.heappush(priority_queue, (initial_f_cost, initial_g_cost, initial_state, [initial_state]))
    
    # cost_so_far dictionary stores the minimum g_cost found to reach a state
    cost_so_far = {initial_state: 0}
    
    print("Starting A* Search...")
    
    while priority_queue:
        f_cost, g_cost, current_state, current_path = heapq.heappop(priority_queue)
        
        _boat_pos, _left, _island, right = current_state
        
        # Goal Check
        if right == goal_state_people:
            print(f"Solution found with {g_cost} moves (optimal).")
            return current_path

        # Optimization: If we found a shorter path to this state already, skip
        # This check is implicitly handled by cost_so_far update logic below,
        # but explicit check after popping can save generating neighbors.
        if g_cost > cost_so_far.get(current_state, float('inf')):
             continue

        # Explore neighbors
        for next_state in get_next_states(current_state, n):
            new_g_cost = g_cost + 1 # Each move costs 1
            
            # If this is a new state or we found a shorter path to it
            if next_state not in cost_so_far or new_g_cost < cost_so_far[next_state]:
                cost_so_far[next_state] = new_g_cost
                h_cost = heuristic(next_state, n)
                new_f_cost = new_g_cost + h_cost
                new_path = current_path + [next_state]
                heapq.heappush(priority_queue, (new_f_cost, new_g_cost, next_state, new_path))
                
    print("No solution found.")
    return None

# --- Example Usage for A* Approach ---
N_COUPLES_ASTAR = 2
solution_path_astar = solve_astar(N_COUPLES_ASTAR)

if solution_path_astar:
    print("\n--- A* Solution Path ---")
    for i, state in enumerate(solution_path_astar):
        boat, left, island, right = state
        print(f"Step {i}:")
        print(f"  Boat: {boat}")
        print(f"  Left Bank : {sorted(list(left))}")
        print(f"  Island    : {sorted(list(island))}")
        print(f"  Right Bank: {sorted(list(right))}")
        print("-" * 20)

Starting A* Search...
Solution found with 10 moves (optimal).

--- A* Solution Path ---
Step 0:
  Boat: L
  Left Bank : ['H1', 'H2', 'W1', 'W2']
  Island    : []
  Right Bank: []
--------------------
Step 1:
  Boat: I
  Left Bank : ['H1', 'W1']
  Island    : ['H2', 'W2']
  Right Bank: []
--------------------
Step 2:
  Boat: L
  Left Bank : ['H1', 'H2', 'W1']
  Island    : ['W2']
  Right Bank: []
--------------------
Step 3:
  Boat: I
  Left Bank : ['W1']
  Island    : ['H1', 'H2', 'W2']
  Right Bank: []
--------------------
Step 4:
  Boat: L
  Left Bank : ['H1', 'W1']
  Island    : ['H2', 'W2']
  Right Bank: []
--------------------
Step 5:
  Boat: I
  Left Bank : []
  Island    : ['H1', 'H2', 'W1', 'W2']
  Right Bank: []
--------------------
Step 6:
  Boat: R
  Left Bank : []
  Island    : ['H1', 'H2']
  Right Bank: ['W1', 'W2']
--------------------
Step 7:
  Boat: I
  Left Bank : []
  Island    : ['H1', 'H2', 'W2']
  Right Bank: ['W1']
--------------------
Step 8:
  Boat: R
  Left Ban

# Some observations
- DFS Result (from your notebook): Your DFS implementation found a solution with 16 moves. DFS explores deeply along one path before backtracking. It guarantees a solution if one exists, but not necessarily the shortest one. The first solution it stumbles upon might be quite long.
- Backtracking Result (from your notebook): Your Backtracking implementation found a solution with 34 moves. Similar to DFS, standard backtracking explores possibilities but doesn't inherently optimize for path length. The solution found depends heavily on the order in which neighbours are explored.
- A* Result (from your notebook): Your A* implementation found a solution with 10 moves. A, when implemented correctly with an admissible heuristic (like the one we used: number of people not on the right bank), guarantees finding the shortest possible path (optimal solution).